<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/delete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install ufal.udpipe

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

import gensim
from ufal.udpipe import Model, Pipeline

In [6]:
!git clone https://github.com/skoltech-nlp/russe_detox_2022

fatal: destination path 'russe_detox_2022' already exists and is not an empty directory.


In [7]:
!wget https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/dev_answers.csv
!wget https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/test_answers.csv

--2022-02-10 20:55:13--  https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/dev_answers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79041 (77K) [text/plain]
Saving to: ‘dev_answers.csv.1’

dev_answers.csv.1   100%[===================>]  77.19K  --.-KB/s    in 0.01s   

2022-02-10 20:55:13 (6.11 MB/s) - ‘dev_answers.csv.1’ saved [79041/79041]

--2022-02-10 20:55:14--  https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/test_answers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [8]:
with open('/content/russe_detox_2022/baselines/delete/toxic_vocab_extended.txt', 'r') as file:
    toxic_words = file.readlines()
toxic_words = [sentence.strip() for sentence in toxic_words]

In [12]:
modelfile = '/content/russe_detox_2022/baselines/delete/udpipe_syntagrus.model'
model_udpipe = Model.load(modelfile)
process_pipeline = Pipeline(model_udpipe, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [13]:
def tokenize(text, tags=False, lemmas=False):
    processed = process_pipeline.process(text)
    content = [l for l in processed.split('\n') if not l.startswith('#')]
    tagged = [w.split('\t') for w in content if w]
    
    tokens = []
    for token in tagged:
        if token[3] == 'PUNCT':
            continue
        
        token_res = ''
        if lemmas:
            token_res = token[2]
        else:
            token_res = token[1]
        if tags:
            token_res += '_' + token[3]
        tokens.append(token_res)
        
    return tokens

In [14]:
df = pd.read_csv('dev_answers.csv', sep='\t')
df.head()

,toxic_comment
0,"Ток температуры горения хватит, чтобы ее распл..."
1,А ты там был.ты вообще служил
2,А сам где кормишься?
3,"этому сайту я давно не доверяю, пишут разную и..."
4,у меня сегодня подобный звонок был. Достали


In [15]:
toxic_inputs_dev = df['toxic_comment'].tolist()

In [16]:
ds = pd.read_csv('test_answers.csv', sep='\t')
ds.head()

,toxic_comment
0,Какой глупый человек эту придумывает
1,В такой ситуации виноваты люди из Ростелекома ...
2,"Актёр может и не плохой,но как человек - не хо..."
3,"Мочите всех, кто нарушает общественный порядок"
4,такие же люди и привели этих людей


In [17]:
toxic_inputs_test = ds['toxic_comment'].tolist()

In [18]:
results_dev = []

for sample in tqdm(toxic_inputs_dev):
    try:
        tokens_lemmas = tokenize(sample, lemmas=True)
    except:
        print(sample)
        tokens_lemmas = sample.split(' ')
    tokens = tokenize(sample, lemmas=False)
    cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        
    results_dev.append(' '.join(cleaned_sentence))

100%|██████████| 800/800 [00:57<00:00, 13.85it/s]


In [19]:
pd.DataFrame({"answer": results_dev})["answer"].apply(pd.Series).to_csv("/content/dev_cleaned.csv", index=False)

In [20]:
results_test = []

for sample in tqdm(toxic_inputs_test):
    try:
        tokens_lemmas = tokenize(sample, lemmas=True)
    except:
        print(sample)
        tokens_lemmas = sample.split(' ')
    tokens = tokenize(sample, lemmas=False)
    cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        
    results_test.append(' '.join(cleaned_sentence))

100%|██████████| 875/875 [00:51<00:00, 16.83it/s]


In [21]:
pd.DataFrame({"answer": results_test})["answer"].apply(pd.Series).to_csv("/content/test_cleaned.csv", index=False)